In [14]:
import os
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# import gc
# gc.disable()

In [2]:
# .mean() - среднее
# максимальная длина ego_id = 13 символов
# как проверить на сколько значим признак в sсikitlear

In [2]:
DATA_DIR = 'data'

### Преобразование датасета train

In [3]:
df_train = pd.read_csv(
    os.path.join(DATA_DIR, 'train.csv'),
    dtype='str',
    encoding='utf-8',
    # usecols=['ego_id', 'u', 'v', 't', 'x2', 'x3'],
    # nrows=1000
)
df_train['ego_id'] = df_train['ego_id'].astype('str')
df_train['u'] = df_train['u'].astype('str')
df_train['v'] = df_train['v'].astype('str')
df_train['t'] = df_train['t'].astype('float').fillna(0.0)
print(len(df_train))
df_train.head(3)

122280372


,ego_id,u,v,t,x1,x2,x3
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0
1,0,135,164,396.7,0.0624627359881627,0.0,0.0
2,0,47,15,0.0,0.0,0.0,1.0


In [4]:
df_attr = pd.read_csv(
    os.path.join(DATA_DIR, 'attr.csv'),
    dtype='str',
    encoding='utf-8',
    # nrows=1000
)
print(len(df_attr))
df_attr.head(3)

14930748


,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1


In [5]:
df = pd.merge(df_train, df_attr, on=['ego_id', 'u'], how='left')
print(len(df))
df

122280372


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1
3,0,5,4,594.5,0.049629738619421,0.0,0.0,49,237065842,1,929914814,900704564
4,0,176,219,45.5,1.2379350629578283,0.0,0.0,21,237065842,1,967669435,149014508
...,...,...,...,...,...,...,...,...,...,...,...,...
122280367,1709396984692,3,5,34.6,2.307749740790665,1.0986122886681098,0.0,16,-1,2,-1,-1
122280368,1709396984692,1,5,53.8,3.729142613395535,3.49650756146648,1.0,23,609282824,2,-1,-1
122280369,1709396984692,1,7,1.5,4.2869843553809766,0.0,0.0,23,609282824,2,-1,-1
122280370,1709396984692,5,11,2.0,3.500756590674839,0.0,0.0,17,-1,2,923370753,-1


In [6]:
del df_train, df_attr

In [7]:
df['node_1'] = df['ego_id'] + '_' + df['u']
df['node_2'] = df['ego_id'] + '_' + df['v']

In [8]:
df['node_1'] = df['node_1'].astype('category').cat.codes
df['node_2'] = df['node_2'].astype('category').cat.codes

In [9]:
df

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1,36,261
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526,40,68
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1,218,53
3,0,5,4,594.5,0.049629738619421,0.0,0.0,49,237065842,1,929914814,900704564,221,217
4,0,176,219,45.5,1.2379350629578283,0.0,0.0,21,237065842,1,967669435,149014508,83,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122280367,1709396984692,3,5,34.6,2.307749740790665,1.0986122886681098,0.0,16,-1,2,-1,-1,5212705,5544904
122280368,1709396984692,1,5,53.8,3.729142613395535,3.49650756146648,1.0,23,609282824,2,-1,-1,5212696,5544904
122280369,1709396984692,1,7,1.5,4.2869843553809766,0.0,0.0,23,609282824,2,-1,-1,5212696,5544906
122280370,1709396984692,5,11,2.0,3.500756590674839,0.0,0.0,17,-1,2,923370753,-1,5212707,5544895


In [11]:
df.to_csv(
    os.path.join(DATA_DIR, 'df_train_attr.csv'),
    index=False,
    encoding='utf-8'
)

In [10]:
df.to_pickle(
    os.path.join(DATA_DIR, 'df_train_attr.pkl')
)

In [13]:
del df

### Catboost

In [3]:
df = pd.read_pickle(
    os.path.join(DATA_DIR, 'df_train_attr.pkl')
)
df

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1,36,261
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526,40,68
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1,218,53
3,0,5,4,594.5,0.049629738619421,0.0,0.0,49,237065842,1,929914814,900704564,221,217
4,0,176,219,45.5,1.2379350629578283,0.0,0.0,21,237065842,1,967669435,149014508,83,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122280367,1709396984692,3,5,34.6,2.307749740790665,1.0986122886681098,0.0,16,-1,2,-1,-1,5212705,5544904
122280368,1709396984692,1,5,53.8,3.729142613395535,3.49650756146648,1.0,23,609282824,2,-1,-1,5212696,5544904
122280369,1709396984692,1,7,1.5,4.2869843553809766,0.0,0.0,23,609282824,2,-1,-1,5212696,5544906
122280370,1709396984692,5,11,2.0,3.500756590674839,0.0,0.0,17,-1,2,923370753,-1,5212707,5544895


In [41]:
train = df[df['ego_id'].isin(list(df['ego_id'].unique())[:1000])]
train = train.dropna(
    subset=['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university'],
    axis=0,
    how='any'
)
train

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1,36,261
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526,40,68
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1,218,53
3,0,5,4,594.5,0.049629738619421,0.0,0.0,49,237065842,1,929914814,900704564,221,217
4,0,176,219,45.5,1.2379350629578283,0.0,0.0,21,237065842,1,967669435,149014508,83,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087470,25769804017,42,85,268.9,0.0,0.0,0.0,34,37139376,2,610438360,-1,5941097,6317790
2087471,25769804017,114,176,214.6,0.0,0.0,0.0,24,884119791,2,-1,-1,5940913,6317612
2087474,25769804017,0,298,221.1,3.746179395837373e-07,0.0,0.0,22,-1,2,-1,-1,5940897,6317728
2087475,25769804017,197,201,239.5,0.0038557969006242,0.0,0.0,20,-1,1,-1,220813102,5941001,6317641


In [42]:
test = df[df['ego_id'].isin(list(df['ego_id'].unique())[1000:1200])]
test = test.dropna(
    subset=['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university'],
    axis=0,
    how='any'
)
test

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
2087478,25769804021,7,14,228.7,0.0001928890294815015,0.0,0.0,32,870128347,2,297359996,656866418,5941306,6317850
2087479,25769804021,101,38,134.9,3.7869421960028085e-09,0.0,0.0,17,-1,2,-1,-1,5941161,6317913
2087480,25769804021,44,35,0.0,0.0,0.0,1.0,29,58668681,2,211809729,461988711,5941278,6317910
2087481,25769804021,49,56,10.1,0.8663829817541147,0.0,0.0,33,443160110,2,-1,-1,5941283,6317931
2087482,25769804021,59,80,189.0,0.8364002693770571,0.0,0.0,27,-1,2,-1,-1,5941294,6317958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453268,25769804565,12,29,53.3,0.0,0.0,1.0,22,551921466,1,-1,-1,5973814,6353529
2453269,25769804565,237,0,18.5,0.1462054840220459,0.0,0.0,36,-1,1,-1,-1,5973906,6353366
2453270,25769804565,89,23,12.2,1.65763185543377,0.0,0.0,16,551921466,1,-1,-1,5973983,6353501
2453271,25769804565,148,135,97.5,0.732958015033126,0.0,0.0,26,551921466,1,-1,-1,5973841,6353404


In [43]:
val = df[df['ego_id'].isin(list(df['ego_id'].unique())[1200:1400])]
val = val.dropna(
    subset=['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university'],
    axis=0,
    how='any'
)
val

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
2453274,25769804566,60,24,0.0,0.0,2.079441541679836,0.0,15,-1,1,-1,-1,5974043,6353621
2453275,25769804566,18,0,0.0,0.0,0.0,1.0,16,-1,1,-1,-1,5974000,6353606
2453276,25769804566,19,42,0.0,0.0,0.0,1.0,-1,-1,-1,-1,-1,5974001,6353640
2453277,25769804566,38,24,0.0,0.0,0.0,1.0,20,361653023,1,744491342,-1,5974018,6353621
2453278,25769804566,30,24,46.0,0.4085968528031649,0.0,0.0,16,361653023,2,678965784,-1,5974012,6353621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2844472,34359738863,188,121,0.0,0.0,0.0,1.0,15,269171552,1,456649666,-1,6604551,7023756
2844473,34359738863,14,1,50.6,1.6746715793041491,0.0,1.0,17,153664763,1,612193720,-1,6604500,7023732
2844474,34359738863,7,51,123.1,8.626947194668843e-05,0.0,0.0,17,-1,1,-1,-1,6604650,7023926
2844475,34359738863,144,20,53.6,0.5415304880867146,0.0,0.0,22,-1,2,-1,-1,6604505,7023841


In [62]:
# features
X = ['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university']
cat_features = ['x3', 'city_id', 'sex', 'school', 'university']
# target (целевой признак)
y = ['x1']

# x2         - числовой
# x3         - категориальный
# age        - числовой
# city       - категориальный
# sex        - категориальный
# school     - категориальный
# university - категориальный

In [63]:
model = CatBoostRegressor(
    cat_features=cat_features,
    eval_metric='RMSE',
    iterations=100,
    depth=10,
    random_seed=48,
    bagging_temperature=0.2,
    od_type='Iter'
)

In [64]:
model.fit(
    train[X], train[y],
    eval_set = (val[X], val[y])
)

Learning rate set to 0.5
0:	learn: 1.0718921	test: 1.0681263	best: 1.0681263 (0)	total: 247ms	remaining: 24.4s
1:	learn: 0.9876542	test: 0.9727669	best: 0.9727669 (1)	total: 460ms	remaining: 22.5s
2:	learn: 0.9627784	test: 0.9440469	best: 0.9440469 (2)	total: 652ms	remaining: 21.1s
3:	learn: 0.9465304	test: 0.9380365	best: 0.9380365 (3)	total: 853ms	remaining: 20.5s
4:	learn: 0.9411754	test: 0.9352689	best: 0.9352689 (4)	total: 1.03s	remaining: 19.6s
5:	learn: 0.9364608	test: 0.9341172	best: 0.9341172 (5)	total: 1.25s	remaining: 19.5s
6:	learn: 0.9344296	test: 0.9335780	best: 0.9335780 (6)	total: 1.45s	remaining: 19.3s
7:	learn: 0.9324750	test: 0.9335507	best: 0.9335507 (7)	total: 1.68s	remaining: 19.3s
8:	learn: 0.9315808	test: 0.9335403	best: 0.9335403 (8)	total: 1.91s	remaining: 19.3s
9:	learn: 0.9308668	test: 0.9336699	best: 0.9335403 (8)	total: 2.16s	remaining: 19.5s
10:	learn: 0.9302893	test: 0.9335745	best: 0.9335403 (8)	total: 2.41s	remaining: 19.5s
11:	learn: 0.9297176	test: 0

In [56]:
model.predict(test[X])

array([0.35768205, 0.59820549, 0.22759127, ..., 0.72185878, 0.49901344,
       0.26043119])

In [57]:
test['pred'] = model.predict(test[X])
test

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2,pred
2087478,25769804021,7,14,228.7,0.0001928890294815015,0.0,0.0,32,870128347,2,297359996,656866418,5941306,6317850,0.357682
2087479,25769804021,101,38,134.9,3.7869421960028085e-09,0.0,0.0,17,-1,2,-1,-1,5941161,6317913,0.598205
2087480,25769804021,44,35,0.0,0.0,0.0,1.0,29,58668681,2,211809729,461988711,5941278,6317910,0.227591
2087481,25769804021,49,56,10.1,0.8663829817541147,0.0,0.0,33,443160110,2,-1,-1,5941283,6317931,0.346705
2087482,25769804021,59,80,189.0,0.8364002693770571,0.0,0.0,27,-1,2,-1,-1,5941294,6317958,0.454480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453268,25769804565,12,29,53.3,0.0,0.0,1.0,22,551921466,1,-1,-1,5973814,6353529,0.384551
2453269,25769804565,237,0,18.5,0.1462054840220459,0.0,0.0,36,-1,1,-1,-1,5973906,6353366,0.556953
2453270,25769804565,89,23,12.2,1.65763185543377,0.0,0.0,16,551921466,1,-1,-1,5973983,6353501,0.721859
2453271,25769804565,148,135,97.5,0.732958015033126,0.0,0.0,26,551921466,1,-1,-1,5973841,6353404,0.499013


In [ ]:
# для submission нужно примерджить сюда данные по фичам из основного датасета и запустить предикт, посмотреть на сколько он совпадает

In [58]:
# from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


# def calc_error(y_true, y_pred):
#     print(mean_absolute_error(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))

# calc_error(
#     test['x1'],
#     test['pred']
# )

0.6602780076911423
309288938005913.2


In [59]:
pd.read_csv('data/submission.csv', dtype='str', encoding='utf-8')

,ego_id,u,v,x1
0,8,0,93,0.000000e+00
1,8,0,143,0.000000e+00
2,8,0,151,1.606742e+00
3,8,1,24,2.649638e-02
4,8,5,4,1.598572e-01
...,...,...,...,...
810971,1709396984676,73,23,1.562170e+00
810972,1709396984676,74,68,4.542098e-01
810973,1709396984676,77,28,7.850433e-02
810974,1709396984676,79,38,9.818121e-01
